![alt text](uspas.png)
# Fundamentals of Accelerator Physics and Technology 
### (with Simulations and Measurements Lab)
# Computer Lab: Transverse Dynamics
##### Author: M. Syphers, E. Harms, N. Neveu

This session deals with transverse dynamics in three sections. The first demonstrates the development of a beam envelope by tracking a particle repeatedly through a FODO structure. The second demonstrates the development of ellipses in phase space for the same FODO structure. The third demonstrates a local orbit distortion in a FODO lattice and how to correct it. 

### Python Notes: 
- Press shift+enter to execute a cell, or use the play button at the top of the window
- Make sure you exectue cells in order, or re-exectue cells if you change something at the top of the notebook.
- You can also exectue the whole notebook by using 'Run all cells' under the 'Run' tab.
- '2**2' represents two squared, i.e. 2^2
- A colon (:) means all values in that dimension. i.e. array[:,2] = all rows, second column
----------

## 1. Betatron Oscillation Envelope

For transverse motion of a particle (beam) to be stable, the trace of the 2X2 transport matrix for each degree of freedom (horizontal and vertical) must be less than or equal to 2 in absolute value: |TrM| ≤2. For a simple FODO structure made up of focusing elements (of focal lengths ±F separated by a distance L) the condition is that F ≥L/2. 

In following cells, the FODO cell parameters are initially F = 20 and L = 25 meters. The worksheet shows eight half-cells, or four full FODO cells, and a particle with initial conditions $x_{0,0}$ and $x’_{0,0}$ is tracked through the system. The indices on the position $x_{i,j}$ are for the j-th passage through the i-th element of the structure. The number of turns begins with $N_{turn}= 1$ and with $x_{0,0}= 5$. 
- Gradually increase $N_{turn}$ to 2, 5, 10, 100 turns. While the particle starts out with a displacement of 5 mm, it ultimately reaches larger ones.

In [ ]:
#Importing the libraries we need
import numpy as np

#Defining some variables
#These will be used later
F      = 20  #
L      = 25  # meters
N_turn = 1
x_00   = 5

**Q1) What is the largest displacement the particle attains in the structure?**


**Q2) Where in the FODO structure i.e. at the F, D, or O (drift) does this maximum displacement occur?**

---
- Return $N_{turn}$ to 1. 
- Adjust the focal length, F, until the parameter $\mu$ is 90°. 
- Change the number of half-cells to Ncell= 20. Look at the plot of the trajectory.

**Q3) How many full cells (50 meter units) does it take for the pattern to repeat itself?**


**Q4) For what value of F does $\mu=60^{\circ}$?**


**Q5) How many full cells (50 meter units) does it take for the pattern to repeat itself?  How would you interpret the parameter $\mu$?**


---

The FODO system will be unstable when |TrM| > 2 or when F < L/2. 
- Leaving the other parameters alone, set F = 12.499. 
- Note that the parameter $\mu$ becomes imaginary! 
- Look at the particle’s trajectory.

**Q6) If the vacuum chamber is 7 centimeters away from the particle’s ideal orbit, how far will the particle travel before it reaches the chamber wall?

---

## 2. Synchrotron Tune vs. Oscillation Amplitude

The two plots below are a trajectory plot plus a phase space plot. As the particle traverses the FODO structure, its position and angle are kept track of and plotted in the phase space plot. For $N_{turn}= 1$, follow and understand how the lines in the two plots are related to one another. A change in slope on the phase space plot corresponds to a “kink” seen in the trajectory plot, etc. 
- Change $N_{turn}$ to 2, 3, 4, 5, 10, and 100 watching the two plots each time. 

**Q7) Print out or make a sketch of this phase space plot and identify with each ellipse its corresponding quadrupole (F or D) and whether it is at the entrance or exit of the magnet.**


In [ ]:
# Phase space plot

---
## 3. Closed Orbit Error & Correction

Today’s final exercise will be to look at how we can correct for a local orbit distortion. In this FODO lattice, F = 20 meters, L  = 25 and at specific locations in the structure (initially position ‘10’), there is an additional element –a steering magnet –which gives the particle an angular deflection of amount $\theta$ each time the particle passes by. By looking at the plot of particle displacement, we see that when the particle starts with x = 0 and x’ = 0, it begins a betatron oscillation when it passes by the steering magnet (see the SOLID trace in the plot).

However, there is a particular orbit which, if the particle trajectory starts out just right, the orbit will be deflected by the steering magnet but when it returns to the beginning of the accelerator will end up with the same position and slope it started out with. Thus, this particular particle will follow the same path over and over again. 
- Change $N_{turn}$ from 1 to 2, 5, 10, 100. 
- Notice how the particle appears to oscillate about the new closed orbit generated by the steering magnet. 

Even though the magnet only steers the particle at one location in the accelerator, it can affect the displacement everywhere. If F and L are in meters, x in millimeters, and x’ in mrad, then $\theta$ is in mrad (0.05 initially).

In [ ]:
#Defining variables
F3 = 20
L3 = 25
N_turns2 = 1


**Q8) What steering error, $\theta_1$ would generate a 3mm ($\pm$ 0.01) maximum displacement of the close orbit in the beam pipe?**


In addition to a steering “error” being defined at position ‘10’ two steering correctors are defined at locations ‘14’ and ‘16’ These two are initially set to zero, and the trajectory is plotted below these definitions. 
- Carefully adjust the strengths of these two correctors so that a particle whose trajectory starts with x = 0, x’ = 0 before $\theta_1$ ends up with x = 0 and x’ = 0 after $\theta_3$. Check the “closure” of your “orbit bump” by changing $N_{turn}$ to some large value and seeing that the orbit indeed repeats itself.

**Q9) For $\theta_1=0.05$, what values of $\theta_2$ and $\theta_3$ are required to bring the trajectory back to x = 0 and x' = 0?**
